In [ ]:
import pandas as pd
import os 

Set Filepath for excel file 

In [ ]:
filepath ='/Users/dnyaneshdesai/Downloads/Govhack2018/atoabsgovhack2018.xlsx'

Read Excel File Worksheet

In [ ]:
excel_file_contents = pd.ExcelFile(filepath)
excel_file_contents.sheet_names

In [ ]:
df_excel_ato_data = excel_file_contents.parse("ATO Data")

Check contents of the dataframe

In [ ]:
df_excel_ato_data.head()

Check if there are null values in any of the columns 

In [ ]:
df_excel_ato_data.isnull().sum()

Filter the columns according to the requirement

In [ ]:
df_excel_ato_data_new = df_excel_ato_data.iloc[:,0:5]

In [ ]:
df_excel_ato_data_new.head()

Create new column to get average income 

In [ ]:
df_excel_ato_data_new['Average Income']= df_excel_ato_data_new['Taxable income or loss1 ']/df_excel_ato_data_new['Individuals1']

Check contents of the dataframe

In [ ]:
df_excel_ato_data_new.head()

In [ ]:
df_excel_ato_data_new.dtypes

Convert float to integer type

In [ ]:
df_excel_ato_data_new['Average Income'] = df_excel_ato_data_new['Average Income'].astype(int)

Check values present in the year column & keep only data related to year 2016

In [ ]:
df_excel_ato_data_new['Income year'].unique()

In [ ]:
intcheck = df_excel_ato_data_new['Income year']==2016

In [ ]:
df_excel_ato_data_new_2016 = df_excel_ato_data_new[intcheck]

In [ ]:
df_excel_ato_data_new_2016.head()

Filter the columns based on the requirement

In [ ]:
file_individual_income = df_excel_ato_data_new_2016[['Postcode', 'Individuals1','Average Income']]

Read the Australian Postcodes file to find the location of each postcode

In [ ]:
filepath_postcodes ='/Users/dnyaneshdesai/Downloads/Govhack2018/australian_postcodes.csv'

In [ ]:
file_postcodes = pd.read_csv(filepath_postcodes)

In [ ]:
file_postcodes.head()

In [ ]:
file_individual_income=file_individual_income.rename(columns = {'Postcode':'postcode'})

Merge the Income data and Postcode data based on the postcode field 

In [ ]:
dtmp = pd.merge(file_individual_income, file_postcodes.drop_duplicates('postcode', keep='first'), on='postcode', how='left')

In [ ]:
dtmp.head()

Filter the data to keep only New South Wales (NSW) related data

In [ ]:
statecheck = dtmp['state']=='NSW'

In [ ]:
final_data_NSW = dtmp[statecheck]

In [ ]:
final_data_NSW.head()

Read the ATO government file again to extract the total number of tax centers in the postcodes in NSW

In [ ]:
df_excel_ato_data = excel_file_contents.parse("Tax Help Center")

In [ ]:
df_excel_ato_data=df_excel_ato_data.rename(columns = {'Post Code':'postcode'})

In [ ]:
df_excel_ato_data.head()

Merge the Income data and Total tax offices data based on the postcode field 

In [ ]:
final_merge = pd.merge(final_data_NSW, df_excel_ato_data.drop_duplicates('postcode', keep='first'), on='postcode', how='left')

In [ ]:
final_merge.head()

Insert the value 0 in the Count column for postcodes which do not have any tax office

In [ ]:
final_merge['Count'] = final_merge['Count'].fillna(0)

In [ ]:
final_merge['Count'] = final_merge['Count'].astype(int)

Filter the columns and then rename based on the requirement

In [ ]:
final_dataset_for_NSW = final_merge[['postcode','Individuals1','Average Income','long','lat','Count']]

In [ ]:
final_dataset_for_NSW=final_dataset_for_NSW.rename(columns = {'postcode':'Postcode'})
final_dataset_for_NSW=final_dataset_for_NSW.rename(columns = {'Individuals1':'Total Tax Payers'})
final_dataset_for_NSW=final_dataset_for_NSW.rename(columns = {'Average Income':'Average Income per Tax Payer'})
final_dataset_for_NSW=final_dataset_for_NSW.rename(columns = {'long':'Longitude'})
final_dataset_for_NSW=final_dataset_for_NSW.rename(columns = {'lat':'Latitude'})
final_dataset_for_NSW=final_dataset_for_NSW.rename(columns = {'Count':'Total Number of Tax Help Offices'})

In [ ]:
final_dataset_for_NSW.head()

In [ ]:
final_dataset_for_NSW.to_csv('/Users/dnyaneshdesai/Downloads/Govhack2018/NSW_Data_for_Exploration.csv', index=False, header=True)

## Code for checking capacity of Individuals served over the years per Tax Centre

In [ ]:
NSW_Postcodes = final_dataset_for_NSW['Postcode']

In [ ]:
NSW_Postcodes = pd.DataFrame(NSW_Postcodes)

In [ ]:
df_excel_ato_data = excel_file_contents.parse("ATO Data")

Merge the dataframes to get the required information based on postcodes

In [ ]:
all_year_data = pd.merge(df_excel_ato_data, NSW_Postcodes.drop_duplicates('Postcode', keep='first'), on='Postcode', how='inner')

Create the new 'Average Income' column for the new dataset

In [ ]:
all_year_data['Average Income']= all_year_data['Taxable income or loss1 ']/all_year_data['Individuals1']

In [ ]:
all_year_data['Average Income'] = all_year_data['Average Income'].astype(int)

Take the necessary columns from the dataframe

In [ ]:
all_year_data = all_year_data[['Postcode','Income year','Average Income','Individuals1',]]

Merge the dataframes based on the basis of postcodes to get the Total tax centers in each area

In [ ]:
all_year_data = pd.merge(all_year_data, final_dataset_for_NSW.drop_duplicates('Postcode', keep='first'), on='Postcode', how='inner')

In [ ]:
all_year_data = all_year_data[['Postcode','Income year','Average Income','Individuals1','Total Number of Tax Help Offices']]

Calculate the number of individuals served by each center every year

In [ ]:
all_year_data['Individuals per Office']= all_year_data['Individuals1']/all_year_data['Total Number of Tax Help Offices']

Convert the float value to integer for the output dataframe

In [ ]:
import numpy as np

In [ ]:
all_year_data = all_year_data.replace(np.inf, 0)

In [ ]:
all_year_data['Individuals per Office'] = all_year_data['Individuals per Office'].astype(int)

In [ ]:
all_year_data.head()

Rename the columns as per requirement 

In [ ]:
all_year_data=all_year_data.rename(columns = {'Individuals1':'Total Tax Payers'})

Write the dataframe to CSV file 

In [ ]:
all_year_data.to_csv('/Users/dnyaneshdesai/Downloads/Govhack2018/NSW_Data_for_all_years_Exploration.csv', index=False, header=True)